In [1]:
import json
from datasets import load_dataset, Dataset
# Specify the path to the JSON file
file_path = 'datasets/squad-sr-lat.json'

# Open the JSON file and load its contents
with open(file_path, 'r', encoding='utf8') as file:
    data = json.load(file)

# Now, 'data' contains the parsed JSON data as a Python dictionary
print(data.keys())


dict_keys(['version', 'data'])


loading the data to wanted format

In [6]:
i = 1
titles = []
contexts = []
queryes = []
for article in data['data']:
    title = article['title']
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        currect_queryes = []
        for qa_pair in paragraph['qas']:
            question = qa_pair['question']
            
            currect_queryes.append(question)
            i += 1
        titles.append(title)
        contexts.append(context)
        queryes.append(currect_queryes)
                # print(answer)

In [7]:
import pandas as pd
# from collections import defaultdict
  
dataset_for_eval_df = pd.DataFrame({'title' : titles,
                                    'context' : contexts,
                                  #  'query_id' : dataset_for_eval['id'],
                                    'queries' : queryes})

dataset_for_eval_df.head()
dataset_for_eval_df['queries'][0]

['Kome se u 1858. godini u Lurdu, Francuska, navodno pojavila Deva Marija?',
 'Šta je ispred Glavne zgrade Notr Dame?',
 'Bazilika Svetog srca u Notr Dami je pored koje strukture?',
 'Šta je Grota u Notr Dami?',
 'Šta se nalazi na vrhu Glavne zgrade u Notr Damu?']

In [8]:
import pickle
import pyarrow as pa
import pyarrow.parquet as pq

dataset_for_eval_df.to_csv('datasets/dataset_processed.csv', index=False)
dataset_for_eval_df.to_pickle('dataset_for_eval.pkl')
table = pa.Table.from_pandas(dataset_for_eval_df)

pq.write_table(table, 'datasets/dataset_processed.parquet')


In [9]:
dataset_for_eval_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18896 entries, 0 to 18895
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    18896 non-null  object
 1   context  18896 non-null  object
 2   queries  18896 non-null  object
dtypes: object(3)
memory usage: 443.0+ KB


making the pipieline for all models

In [10]:
# Create model embeddings
# def generate_embeddings(text):
# Initialize the question-answering pipeline
# Measuring query-context similarity
# def compute_similarity(query, context, top_k):
# Generate predictions

In [149]:
import pandas as pd
import torch
import numpy as np
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModel
from collections import defaultdict
import pytrec_eval
import json
from sklearn.metrics.pairwise import cosine_similarity



1. ucitati model i tokenizer
2. ucitati df
3. prolazimo kroz sve recorde u df i za svaki record
    1. ubacimo listu query u uciran model i dobije listu embedinga
    2. ubacimo context u model i dobijemo encodirano
    3. merimo slicnost izmedju embedinga i matrice vecContexta@matriceVektoraQuerya
    pored cosinusne se cuva i index contexta
    nama treba i za query da se cuva koji je njegog paragraf
    4. sortiramo sve umnoske, iyvucemo najslicniji

In [15]:
loaded_table = pq.read_table("datasets/dataset_processed.parquet")
df = loaded_table.to_pandas()
df.head()

,title,context,queries
0,University_of_Notre_Dame,"Arhitektonski, škola ima katolički karakter. N...","[Kome se u 1858. godini u Lurdu, Francuska, na..."
1,University_of_Notre_Dame,"Kao i na većini drugih univerziteta, studenti ...",[Kada je počelo izdavanje Scholastičkog časopi...
2,University_of_Notre_Dame,Univerzitet je glavna sedište Kongregacije Sve...,"[Gde je sedište Kongregacije Svetog Krsta?, Št..."
3,University_of_Notre_Dame,Koledž za inženjerstvo osnovan je 1920. godine...,[Koliko diploma na nivo BS se nudi u Koledžu z...
4,University_of_Notre_Dame,Svi studenti Notr Dame-a su deo jednog od pet ...,[Koje su subjekti koje pružaju pomoć u upravlj...


In [17]:
#mead puling
def get_sentence_embedding(sentence, model, tokenizer): 
    """""
    Pitanja:
    - Da li koristiti mean pooling ili CSL pooling?
    - U SQuAD datasetu su queries string i treba ih parsirati u listu
    """""
    encoded_input = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
    attention_mask = encoded_input['attention_mask']   # to indicate which tokens are valid and which are padding
    
    # Get the model output (without the specific classification head)
    with torch.no_grad():
        output = model(**encoded_input)

    token_embeddings = output.last_hidden_state # poslednji nivo
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()

    # mean pooling operation, considering the BERT input_mask and padding
    sentence_embedding = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
    return sentence_embedding.flatten().tolist()
    # return sentence_embedding


def cos_sim(context_embed, query_embed):
    return context_embed @ query_embed.T


def evaluate(model_name, dataset_path):
    
    loaded_table = pq.read_table("datasets/dataset_processed.parquet")
    df = loaded_table.to_pandas()
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    ground_truth = {}# ovo je nase
    # qrel = {
    #     "tekst_query1":
    #     {"1_id_paragrafa": 0,
    #     "2_id_paragrafa": 1,
    #     "3_id_paragrafa": 0},
    #     "tekst_query2":
    #     {"1_id_paragrafa": 1,
    #     "2_id_paragrafa": 0,
    #     "3_id_paragrafa": 0},
    # }
    retrieved_scores = {}  # ovo su cosinusne

    #we embed the contexts
    context_embeddings = {}
    for idx, row in df.iterrows():
        if idx >= 2:  # Break the loop after two iterations
            break
        context = row['context']
        embedded_context = get_sentence_embedding(context, model, tokenizer)
        context_embeddings[str(idx)] = embedded_context
        # print(idx)
    
    # we fill the dict used for metrics
    for idx, row in df.iterrows():
        if idx >= 2:  # Break the loop after two iterations
            break
        queries = row['queries'] # [1:-1].split("', ' ") zato sto je sacivan kao string pa mora u listu da se pretvara
        # print(queries)
        # print(type(queries))
        for query in queries:
            query = query.strip()
            # print(query)
            ground_truth[query] = {}
            retrieved_scores[query] = {}
            embedded_query = get_sentence_embedding(query, model, tokenizer)
            for current_context_id, current_context_embedding in context_embeddings.items():
                ground_truth[query][current_context_id] = 0
                # score = cos_sim(current_context_embedding, embedded_query)
                # print(embedded_query)
                score = cosine_similarity(np.array(current_context_embedding).reshape(1, -1), np.array(embedded_query).reshape(1, -1))
                # print(score.item())
                retrieved_scores[query][current_context_id] = score.item()
            ground_truth[query][str(idx)] = 1 # samo ovaj kome pripada je 1 ostali su nula, necemo da punimo jer je default dict
            
    
    evaluator = pytrec_eval.RelevanceEvaluator(
    ground_truth, {'map', 'ndcg'}
    ) 
    # print(context_embeddings)
    print(json.dumps(evaluator.evaluate(retrieved_scores), indent=1))
    # evaluator.evaluate(run)
    return retrieved_scores
     




run = evaluate("classla/bcms-bertic", 'datasets/dataset_processed.csv')

/home/selena/sele/SRBedding/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{
 "Kome se u 1858. godini u Lurdu, Francuska, navodno pojavila Deva Marija?": {
  "map": 1.0,
  "ndcg": 1.0
 },
 "\u0160ta je ispred Glavne zgrade Notr Dame?": {
  "map": 1.0,
  "ndcg": 1.0
 },
 "Bazilika Svetog srca u Notr Dami je pored koje strukture?": {
  "map": 1.0,
  "ndcg": 1.0
 },
 "\u0160ta je Grota u Notr Dami?": {
  "map": 1.0,
  "ndcg": 1.0
 },
 "\u0160ta se nalazi na vrhu Glavne zgrade u Notr Damu?": {
  "map": 1.0,
  "ndcg": 1.0
 },
 "Kada je po\u010delo izdavanje Scholasti\u010dkog \u010dasopisa Notr dame?": {
  "map": 0.5,
  "ndcg": 0.6309297535714575
 },
 "Koliko \u010desto se \"Notr Dams d\u017eongler\" objavljuje?": {
  "map": 0.5,
  "ndcg": 0.6309297535714575
 },
 "Kako se zove dnevni studentski list u Notr Damu?": {
  "map": 0.5,
  "ndcg": 0.6309297535714575
 },
 "Koliko studentskih novinara se nalazi na Notr Dame?": {
  "map": 0.5,
  "ndcg": 0.6309297535714575
 },
 "U kojoj godini je studentski list \"Sporo\u010dni razum\" po\u010deo da se objavljuje na Notr Damu

In [14]:
print(run)


{'Kome se u 1858. godini u Lurdu, Francuska, navodno pojavila Deva Marija?': {'0': 0.9702974328351258, '1': 0.9417458116887851}, 'Šta je ispred Glavne zgrade Notr Dame?': {'0': 0.9398622207027303, '1': 0.9148289297375717}, 'Bazilika Svetog srca u Notr Dami je pored koje strukture?': {'0': 0.9571723517991639, '1': 0.9233963285326181}, 'Šta je Grota u Notr Dami?': {'0': 0.9302698055098331, '1': 0.8982230382425597}, 'Šta se nalazi na vrhu Glavne zgrade u Notr Damu?': {'0': 0.97306117336876, '1': 0.9459882890289258}, 'Kada je počelo izdavanje Scholastičkog časopisa Notr dame?': {'0': 0.9430615139247354, '1': 0.909179445095098}, 'Koliko često se "Notr Dams džongler" objavljuje?': {'0': 0.9401447997890098, '1': 0.9091041988650693}, 'Kako se zove dnevni studentski list u Notr Damu?': {'0': 0.9681517831548883, '1': 0.9366752558501559}, 'Koliko studentskih novinara se nalazi na Notr Dame?': {'0': 0.9355253300773242, '1': 0.9058199066693698}, 'U kojoj godini je studentski list "Sporočni razum" p

In [ ]:
# sentence transformer

# batching u sentence transformerima i trnasformerima, 

# embedowan query i array svih conteksta u 1 calling, sortiraju se, top k, dobiju se indexi

#clod

### Second try

In [1]:
from sentence_transformers import SentenceTransformer

# Load the model as a SentenceTransformer
model = SentenceTransformer("classla/bcms-bertic")


/home/selena/sele/SRBedding/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
No sentence-transformers model found with name classla/bcms-bertic. Creating a new one with mean pooling.
/home/selena/sele/SRBedding/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [59]:
class HybridPooling(torch.nn.Module):
    def __init__(self, word_embedding_dimension):
        super(HybridPooling, self).__init__()
        self.word_embedding_dimension = word_embedding_dimension

    def forward(self, features):
        token_embeddings = features['token_embeddings']
        attention_mask = features['attention_mask']

        # Mean Pooling
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        mean_pooling = sum_embeddings / sum_mask

        # Max Pooling
        max_pooling = torch.max(token_embeddings * input_mask_expanded, 1)[0]

        # Concatenate Mean and Max Pooling
        return torch.cat((mean_pooling, max_pooling), 1)
    
class AttentionPooling(torch.nn.Module):
    def __init__(self, word_embedding_dimension):
        super(AttentionPooling, self).__init__()
        self.attention = torch.nn.Linear(word_embedding_dimension, 1)

    def forward(self, features):
        token_embeddings = features['token_embeddings']  # Shape: (batch_size, seq_len, hidden_dim)
        attention_mask = features['attention_mask']      # Shape: (batch_size, seq_len)

        # Apply the attention mechanism
        attention_scores = self.attention(token_embeddings).squeeze(-1)  # Shape: (batch_size, seq_len)
        
        # Mask the attention scores so that padded tokens have no influence
        attention_scores = attention_scores.masked_fill(attention_mask == 0, -1e9)  # Shape: (batch_size, seq_len)
        
        # Softmax over the sequence length dimension to get attention weights
        attention_weights = torch.nn.functional.softmax(attention_scores, dim=1)  # Shape: (batch_size, seq_len)
        
        # Apply attention weights to the token embeddings
        attention_weights = attention_weights.unsqueeze(-1)  # Shape: (batch_size, seq_len, 1)
        weighted_sum = torch.sum(token_embeddings * attention_weights, dim=1)  # Shape: (batch_size, hidden_dim)

        return weighted_sum
    
class AttentionHybridPooling(torch.nn.Module):
    def __init__(self, word_embedding_dimension):
        super(AttentionHybridPooling, self).__init__()
        self.attention = torch.nn.Linear(word_embedding_dimension, 1)

    def forward(self, features):
        token_embeddings = features['token_embeddings']  # (batch_size, seq_len, hidden_dim)
        attention_mask = features['attention_mask']      # (batch_size, seq_len)

        # Apply attention mechanism
        attention_scores = self.attention(token_embeddings).squeeze(-1)  # (batch_size, seq_len)
        attention_scores = attention_scores.masked_fill(attention_mask == 0, -1e9)
        attention_weights = torch.nn.functional.softmax(attention_scores, dim=1)  # (batch_size, seq_len)
        
        # Apply attention weights to the token embeddings
        attention_pooling = torch.sum(token_embeddings * attention_weights.unsqueeze(-1), dim=1)  # (batch_size, hidden_dim)

        # Mean Pooling
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, dim=1)
        sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
        mean_pooling = sum_embeddings / sum_mask  # (batch_size, hidden_dim)

        # Max Pooling
        max_pooling, _ = torch.max(token_embeddings * input_mask_expanded, dim=1)  # (batch_size, hidden_dim)

        # Concatenate Attention, Mean, and Max Pooling results
        combined_pooling = torch.cat([attention_pooling, mean_pooling, max_pooling], dim=1)  # (batch_size, hidden_dim * 3)

        return combined_pooling


In [165]:
import torch
from transformers import AutoModel
from sentence_transformers import SentenceTransformer, models
from sklearn.metrics.pairwise import cosine_similarity



model = AutoModel.from_pretrained("classla/bcms-bertic")

# Combine the model and pooling into a SentenceTransformer
word_embedding_model = models.Transformer(model_name_or_path="classla/bcms-bertic")
pooling_model = models.Pooling(word_embedding_dimension=model.config.hidden_size, pooling_mode_cls_token=True, pooling_mode_mean_tokens=False)
# pooling_model = HybridPooling(word_embedding_dimension=model.config.hidden_size)
# pooling_model = AttentionHybridPooling(word_embedding_dimension=model.config.hidden_size)
# pooling_model = models.Pooling(word_embedding_dimension=model.config.hidden_size, pooling_mode="weightedmean")
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
# layers = models.WeightedLayerPooling(word_embedding_model.get_word_embedding_dimension(),num_hidden_layers = 24, layer_start= 12)
sentence_transformer_model = SentenceTransformer(modules=[word_embedding_model,  pooling_model])
# sentence_transformer_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
# sentence_transformer_model._modules["1"].pooling_mode_mean_tokens = False
# sentence_transformer_model._modules["1"].pooling_mode_cls_token = True
# Now, you can use `sentence_transformer_model` for sentence embeddings
sentence = "This is a test sentence."
encoded_input = sentence_transformer_model.encode(sentence)
print(len(encoded_input))


768


In [156]:
# word_embedding_model = AutoModel.from_pretrained("classla/bcms-bertic")
# # word_embedding_model.auto_model = zs.model.roberta
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
# layers = models.WeightedLayerPooling(word_embedding_model.get_word_embedding_dimension(),num_hidden_layers = 24, layer_start= 12)
# model = SentenceTransformer(modules=[word_embedding_model,layers,  pooling_model], device = 0)
# model.encode("An example")

In [157]:
# embed_model ="classla/bcms-bertic"
# word_embedding_model = models.Transformer(embed_model)
#             # word_embedding_model.auto_model = zs.model.roberta
# pooling_model = models.Pooling(
#                     word_embedding_model.get_word_embedding_dimension())
# print(word_embedding_model.get_word_embedding_dimension())
# layers = models.WeightedLayerPooling(
#     word_embedding_model.get_word_embedding_dimension(), num_hidden_layers=12)  # 
# embedder = SentenceTransformer(
#     modules=[word_embedding_model,pooling_model])
# a = embedder.encode("An example")
# embedder2 = SentenceTransformer(
#     modules=[word_embedding_model, pooling_model,layers])
# b = embedder2.encode("An example")
# print(a[0],b[0])

In [168]:
from sklearn.metrics.pairwise import cosine_similarity

sentence_transformer_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

sentence1 = "Glavni međunarodni aerodrom je Nikola Tesla."
sentence2 = "Glupost."

encoded_input1 = sentence_transformer_model.encode(sentence1)
encoded_input2 = sentence_transformer_model.encode(sentence2)

# Reshape the embeddings to 2D arrays
encoded_input1 = np.array(encoded_input1).reshape(1, -1)
encoded_input2 = np.array(encoded_input2).reshape(1, -1)

# Calculate cosine similarity
similarity = cosine_similarity(encoded_input1, encoded_input2)[0][0]

print(f"Cosine Similarity: {similarity:.4f}")


/home/selena/sele/SRBedding/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Cosine Similarity: 0.2349


In [169]:
sentence1 = "this is a great laptop"
sentence2 = "yesterday I saw Marrie at the park."

encoded_input1 = sentence_transformer_model.encode(sentence1)
encoded_input2 = sentence_transformer_model.encode(sentence2)

# Reshape the embeddings to 2D arrays
encoded_input1 = np.array(encoded_input1).reshape(1, -1)
encoded_input2 = np.array(encoded_input2).reshape(1, -1)

# Calculate cosine similarity
similarity = cosine_similarity(encoded_input1, encoded_input2)[0][0]

print(f"Cosine Similarity: {similarity:.4f}")


Cosine Similarity: -0.0831


In [170]:
sentence1 = "this is a great laptop"
sentence2 = "this laptop is great"

encoded_input1 = sentence_transformer_model.encode(sentence1)
encoded_input2 = sentence_transformer_model.encode(sentence2)

# Reshape the embeddings to 2D arrays
encoded_input1 = np.array(encoded_input1).reshape(1, -1)
encoded_input2 = np.array(encoded_input2).reshape(1, -1)

# Calculate cosine similarity
similarity = cosine_similarity(encoded_input1, encoded_input2)[0][0]

print(f"Cosine Similarity: {similarity:.4f}")


Cosine Similarity: 0.9548


In [20]:
import random
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from datasets import load_dataset

# Load a model
# model = SentenceTransformer('all-mpnet-base-v2')
word_embedding_model = models.Transformer(model_name_or_path="classla/bcms-bertic")
pooling_model = models.Pooling(word_embedding_dimension=model.config.hidden_size, pooling_mode_mean_tokens=True)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


# Load the Quora IR dataset (https://huggingface.co/datasets/BeIR/quora, https://huggingface.co/datasets/BeIR/quora-qrels)
corpus = load_dataset("BeIR/quora", "corpus", split="corpus")
queries = load_dataset("BeIR/quora", "queries", split="queries")
relevant_docs_data = load_dataset("BeIR/quora-qrels", split="validation")

# Shrink the corpus size heavily to only the relevant documents + 10,000 random documents
required_corpus_ids = list(map(str, relevant_docs_data["corpus-id"]))
required_corpus_ids += random.sample(corpus["_id"], k=10_000)
corpus = corpus.filter(lambda x: x["_id"] in required_corpus_ids)

# Convert the datasets to dictionaries
corpus = dict(zip(corpus["_id"], corpus["text"]))  # Our corpus (cid => document)
queries = dict(zip(queries["_id"], queries["text"]))  # Our queries (qid => question)
relevant_docs = {}  # Query ID to relevant documents (qid => set([relevant_cids])
for qid, corpus_ids in zip(relevant_docs_data["query-id"], relevant_docs_data["corpus-id"]):
    qid = str(qid)
    corpus_ids = str(corpus_ids)
    if qid not in relevant_docs:
        relevant_docs[qid] = set()
    relevant_docs[qid].add(corpus_ids)

# Given queries, a corpus and a mapping with relevant documents, the InformationRetrievalEvaluator computes different IR metrics.
ir_evaluator = InformationRetrievalEvaluator(
    queries=queries,
    corpus=corpus,
    relevant_docs=relevant_docs,
    name="BeIR-quora-dev",
)
results = ir_evaluator(model)
'''
Information Retrieval Evaluation of the model on the BeIR-quora-dev dataset:
Queries: 5000
Corpus: 17476

Score-Function: cosine
Accuracy@1: 96.26%
Accuracy@3: 99.38%
Accuracy@5: 99.74%
Accuracy@10: 99.94%
Precision@1: 96.26%
Precision@3: 43.01%
Precision@5: 27.66%
Precision@10: 14.58%
Recall@1: 82.93%
Recall@3: 96.28%
Recall@5: 98.38%
Recall@10: 99.55%
MRR@10: 0.9782
NDCG@10: 0.9807
MAP@100: 0.9732
Score-Function: dot
Accuracy@1: 96.26%
Accuracy@3: 99.38%
Accuracy@5: 99.74%
Accuracy@10: 99.94%
Precision@1: 96.26%
Precision@3: 43.01%
Precision@5: 27.66%
Precision@10: 14.58%
Recall@1: 82.93%
Recall@3: 96.28%
Recall@5: 98.38%
Recall@10: 99.55%
MRR@10: 0.9782
NDCG@10: 0.9807
MAP@100: 0.9732
'''
print(ir_evaluator.primary_metric)
# => "BeIR-quora-dev_cosine_map@100"
print(results[ir_evaluator.primary_metric])
# => 0.9732046108457585

/home/selena/sele/SRBedding/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Filter:   0%|          | 0/522931 [00:00<?, ? examples/s]

BeIR-quora-dev_cosine_map@100
0.48346267444778823


In [23]:
print(results)

{'BeIR-quora-dev_cosine_accuracy@1': 0.474, 'BeIR-quora-dev_cosine_accuracy@3': 0.5604, 'BeIR-quora-dev_cosine_accuracy@5': 0.597, 'BeIR-quora-dev_cosine_accuracy@10': 0.6398, 'BeIR-quora-dev_cosine_precision@1': np.float64(0.474), 'BeIR-quora-dev_cosine_precision@3': np.float64(0.2081333333333333), 'BeIR-quora-dev_cosine_precision@5': np.float64(0.13808), 'BeIR-quora-dev_cosine_precision@10': np.float64(0.07672000000000001), 'BeIR-quora-dev_cosine_recall@1': np.float64(0.4063639496614497), 'BeIR-quora-dev_cosine_recall@3': np.float64(0.49620386098541985), 'BeIR-quora-dev_cosine_recall@5': np.float64(0.5341236293673647), 'BeIR-quora-dev_cosine_recall@10': np.float64(0.577914407847055), 'BeIR-quora-dev_cosine_ndcg@10': np.float64(0.514082424479835), 'BeIR-quora-dev_cosine_mrr@10': 0.5268692063492061, 'BeIR-quora-dev_cosine_map@100': np.float64(0.48346267444778823), 'BeIR-quora-dev_dot_accuracy@1': 0.0, 'BeIR-quora-dev_dot_accuracy@3': 0.001, 'BeIR-quora-dev_dot_accuracy@5': 0.002, 'BeIR

In [14]:
print(corpus)

{'68': 'How is the average speed of gas molecules determined?', '101': 'Is Career Launcher good for RBI Grade B preparation?', '117': "I was suddenly logged off Gmail. I can't remember my Gmail password and just realized the recovery email is no longer alive. What can I do?", '118': "I can't remember my Gmail password or my recovery email. How can I recover my e-mail?", '143': 'What is a narcissistic personality disorder?', '144': 'What is narcissistic personality disorder?', '182': 'Which book should I choose for reference for physics and chemistry (class 11, CBSE board)?', '233': 'Why do some girls like to stick their their tongues out when taking pictures?', '248': 'What is the greatest mystery of all time?', '287': 'Does a black hole have mass?', '317': 'What does the Quora website look like to members of Quora moderation?', '351': 'Which is the best fixed income fund?', '377': 'Is it possible to pursue many different things in life?', '380': 'No fanboys please, but who was the tru

In [16]:
print(queries)

{'318': 'How does Quora look to a moderator?', '378': 'How do I refuse to chose between different things to do in my life?', '379': 'Did Ben Affleck shine more than Christian Bale as Batman?', '399': 'What are the effects of demonitization of 500 and 1000 rupees notes on real estate sector?', '420': 'Why creativity is important?', '540': 'How can I get free gems in Clash of Clans?', '548': 'How do you feel when someone upvotes your answer on Quora?', '609': 'What are the best thriller movie in Hollywood?', '744': 'What should someone do to overcome anxiety?', '784': 'Why should one hate Shahrukh Khan?', '858': 'What would Hillary Clinton do now that the election is over?', '975': 'What are the best ways to prepare for software interviews?', '1079': 'How do I become a good computer science engineer?', '1088': 'What does it feel like to be beautiful?', '1164': 'Do Trump voters care that he has reneged all his campaign promises?', '1166': 'Can I learn while sleeping? If so, how?', '1248':

In [17]:
print(relevant_docs)

{'318': {'317'}, '378': {'377'}, '379': {'29976', '45646', '261886', '380', '45647'}, '399': {'3955', '18738', '55150', '50270', '21662', '2607', '55149', '58672', '276497', '1780', '95146', '102346', '22099', '3956', '364917', '9754', '21663', '15261', '44346', '172795', '53404', '2606', '75211', '1779', '102345', '400', '100513', '78756'}, '420': {'419'}, '540': {'93765', '541'}, '548': {'549'}, '609': {'26710', '608'}, '744': {'54251', '745', '82405', '257345', '33905', '17190', '62075', '23949', '82404', '17191'}, '784': {'44095', '785'}, '858': {'137433', '859'}, '975': {'974'}, '1079': {'1078', '260818'}, '1088': {'40625', '40626'}, '1164': {'98063', '30996', '227207', '30997', '98062', '509713', '1165'}, '1166': {'1167'}, '1248': {'39021', '46839', '39022', '94479', '347497', '137904', '224657', '27059', '334904', '74599', '46840', '224085', '27060', '109328', '1249', '141532'}, '1350': {'1349', '331457'}, '1453': {'1452', '249576', '164807'}, '1578': {'1577'}, '1803': {'1804'},

In [88]:
import pyarrow as pa
import pyarrow.parquet as pq

def get_data_for_evaluation(dataset_name):
    loaded_table = pq.read_table(dataset_name)
    df = loaded_table.to_pandas()
    corpus = {}
    queries = {}
    relevant_docs = {}
    query_idx = 1
    for idx, row in df.iterrows():
        if idx >= 1000:  # Break the loop after two iterations
            break
        corpus[idx] = row['context']
        for query in row['queries']:
            query = query.strip()
            queries[query_idx] = query
            if query_idx not in relevant_docs:
                relevant_docs[query_idx] = set()
            relevant_docs[query_idx].add(idx)
            query_idx += 1
    return queries, corpus, relevant_docs

        # print(idx)

queries, corpus, relevant_docs = get_data_for_evaluation("datasets/dataset_processed.parquet")


In [72]:
queries

{1: 'Kome se u 1858. godini u Lurdu, Francuska, navodno pojavila Deva Marija?',
 2: 'Šta je ispred Glavne zgrade Notr Dame?',
 3: 'Bazilika Svetog srca u Notr Dami je pored koje strukture?',
 4: 'Šta je Grota u Notr Dami?',
 5: 'Šta se nalazi na vrhu Glavne zgrade u Notr Damu?',
 6: 'Kada je počelo izdavanje Scholastičkog časopisa Notr dame?',
 7: 'Koliko često se "Notr Dams džongler" objavljuje?',
 8: 'Kako se zove dnevni studentski list u Notr Damu?',
 9: 'Koliko studentskih novinara se nalazi na Notr Dame?',
 10: 'U kojoj godini je studentski list "Sporočni razum" počeo da se objavljuje na Notr Damu?'}

In [76]:
corpus

{0: 'Arhitektonski, škola ima katolički karakter. Na vrhu zlatne kupole Glavne zgrade nalazi se zlatna statua Deve Marije. Neposredno ispred Glavne zgrade i ispred nje, nalazi se bakarna statua Hrista sa podignutim rukama sa legendom "Venite ad Me Omnes". Naokolo Glavne zgrade nalazi se Bazilika Svetog Srca. Neposredno iza bazilike nalazi se Grota, marijano mesto molitve i razmišljanja. To je replika pećine u Lurdesu, Francuska, gde se Deva Marija navodno pojavila Svetoj Bernadet Subiru 1858. godine. Na kraju glavnog voza (i u direktnoj liniji koja se povezuje kroz 3 statue i Zlatnu kupolu), nalazi se jednostavna, moderna kamenna statuja Marije.',
 1: 'Kao i na većini drugih univerziteta, studenti Notr Dama vode brojne medijske izvore. Devet studenata-upravljaju izlaza uključuju tri novine, i radio i televizijska stanica, i nekoliko časopisa i časopisa. Počeo je kao jednostranični časopis u septembru 1876. godine, časopis Scholastic izdaje se dva puta mesečno i tvrdi da je najstarija k

In [74]:
relevant_docs

{1: {0},
 2: {0},
 3: {0},
 4: {0},
 5: {0},
 6: {1},
 7: {1},
 8: {1},
 9: {1},
 10: {1}}

In [90]:
from transformers import AutoModel
from sentence_transformers import SentenceTransformer, models
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction
import pandas as pd
import torch
import numpy as np
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModel
from collections import defaultdict
import pytrec_eval
import json
from sklearn.metrics.pairwise import cosine_similarity



def load_sentence_tranformer_from_transformer(model_name):
    model = AutoModel.from_pretrained(model_name)

    # Combine the model and pooling into a SentenceTransformer
    word_embedding_model = models.Transformer(model_name_or_path=model_name)
    pooling_model = models.Pooling(word_embedding_dimension=model.config.hidden_size, pooling_mode_mean_tokens=True)
    return SentenceTransformer(modules=[word_embedding_model, pooling_model])

def evaluate(model_name, dataset_name):
    model = load_sentence_tranformer_from_transformer(model_name)

    queries, corpus, relevant_docs = get_data_for_evaluation(dataset_name)
    ir_evaluator = InformationRetrievalEvaluator(
    queries=queries,
    corpus=corpus,
    relevant_docs=relevant_docs,
    name=f"{model_name}-{dataset_name}-evlatuation",
    write_csv=True
    )
    results = ir_evaluator(model)
    print(ir_evaluator.primary_metric)
    print(results[ir_evaluator.primary_metric])
    return results

datasets = ["datasets/dataset_processed.parquet"]
models_ = ["classla/bcms-bertic"]

for dataset_name, model_name in zip(datasets, models_):
    # print(dataset_name)
    # print(model_name)
    print(evaluate(model_name=model_name, dataset_name=dataset_name))


/home/selena/sele/SRBedding/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


classla/bcms-bertic-datasets/dataset_processed.parquet-evlatuation_cosine_map@100
0.013965753695357182
{'classla/bcms-bertic-datasets/dataset_processed.parquet-evlatuation_cosine_accuracy@1': 0.005184033177812338, 'classla/bcms-bertic-datasets/dataset_processed.parquet-evlatuation_cosine_accuracy@3': 0.011059270779332988, 'classla/bcms-bertic-datasets/dataset_processed.parquet-evlatuation_cosine_accuracy@5': 0.01745291169863487, 'classla/bcms-bertic-datasets/dataset_processed.parquet-evlatuation_cosine_accuracy@10': 0.026438569206842923, 'classla/bcms-bertic-datasets/dataset_processed.parquet-evlatuation_cosine_precision@1': np.float64(0.005184033177812338), 'classla/bcms-bertic-datasets/dataset_processed.parquet-evlatuation_cosine_precision@3': np.float64(0.0036864235931109955), 'classla/bcms-bertic-datasets/dataset_processed.parquet-evlatuation_cosine_precision@5': np.float64(0.0034905823397269747), 'classla/bcms-bertic-datasets/dataset_processed.parquet-evlatuation_cosine_precision@

### ms marco

In [95]:
from datasets import load_dataset

marco_ds = load_dataset("microsoft/ms_marco", "v1.1")
data_test = marco_ds['test']
ms_marco = data_test.select_columns(['passages', 'query', 'query_id'])
ms_marco

Dataset({
    features: ['passages', 'query', 'query_id'],
    num_rows: 9650
})

In [98]:
data_test[0]['passages']

{'is_selected': [0, 0, 1, 0, 0, 0, 0],
 'passage_text': ['We have been feeding our back yard squirrels for the fall and winter and we noticed that a few of them have missing fur. One has a patch missing down his back and under both arms. Also another has some missing on his whole chest. They are all eating and seem to have a good appetite.',
  'Critters cannot stand the smell of human hair, so sprinkling a barrier of hair clippings around your garden, or lightly working it into the soil when you plant bulbs, apparently does have some merit. The whole thing kind of makes me laugh. It never occurred to me that we are the ones that stink.',
  "Spread some human hair around your vegetable and flower gardens. This will scare the squirrels away because humans are predators of squirrels. It is better if the hair hasn't been washed so the squirrels will easily pick up the human scent.",
  '1 You can sprinkle blood meal around your garden as well. 2  Don’t trap and relocate squirrels. 3  This i

In [92]:
print(len(ms_marco["query_id"]))
print(len(ms_marco.unique('query_id')))

9650
9650


In [105]:
titles = []
queries = []
contexts = []
for index, example in enumerate(data_test):
    # Access individual fields like 'query', 'query_id', and 'passages'
    query = example['query']
    query_id = example['query_id']
    is_selected = example['passages']['is_selected']
    passage_text = example['passages']['passage_text']
    for i in range(len(is_selected)):
        titles.append("")
        if is_selected[i] == 1:
            queries.append([query])
            contexts.append(passage_text[i])
        else:
            queries.append([])
            contexts.append(passage_text[i])

print()


In [106]:
import pandas as pd
# from collections import defaultdict
  
dataset_for_eval_df = pd.DataFrame({'title' : titles,
                                    'context' : contexts,
                                  #  'query_id' : dataset_for_eval['id'],
                                    'queries' : queries})

dataset_for_eval_df.head()

,title,context,queries
0,,We have been feeding our back yard squirrels f...,[]
1,,"Critters cannot stand the smell of human hair,...",[]
2,,Spread some human hair around your vegetable a...,[does human hair stop squirrels]
3,,1 You can sprinkle blood meal around your gard...,[]
4,,Hair loss in squirrels can be caused by mange ...,[]


In [107]:
table = pa.Table.from_pandas(dataset_for_eval_df)

pq.write_table(table, 'datasets/dataset_processed_marco.parquet')

In [108]:
datasets = ["datasets/dataset_processed.parquet", 'datasets/dataset_processed_marco.parquet']
models_ = ["classla/bcms-bertic"]

for dataset_name, model_name in zip(datasets, models_):
    print(evaluate(model_name=model_name, dataset_name=dataset_name))


/home/selena/sele/SRBedding/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


classla/bcms-bertic-datasets/dataset_processed.parquet-evlatuation_cosine_map@100
0.013965753695357182
{'classla/bcms-bertic-datasets/dataset_processed.parquet-evlatuation_cosine_accuracy@1': 0.005184033177812338, 'classla/bcms-bertic-datasets/dataset_processed.parquet-evlatuation_cosine_accuracy@3': 0.011059270779332988, 'classla/bcms-bertic-datasets/dataset_processed.parquet-evlatuation_cosine_accuracy@5': 0.01745291169863487, 'classla/bcms-bertic-datasets/dataset_processed.parquet-evlatuation_cosine_accuracy@10': 0.026438569206842923, 'classla/bcms-bertic-datasets/dataset_processed.parquet-evlatuation_cosine_precision@1': np.float64(0.005184033177812338), 'classla/bcms-bertic-datasets/dataset_processed.parquet-evlatuation_cosine_precision@3': np.float64(0.0036864235931109955), 'classla/bcms-bertic-datasets/dataset_processed.parquet-evlatuation_cosine_precision@5': np.float64(0.0034905823397269747), 'classla/bcms-bertic-datasets/dataset_processed.parquet-evlatuation_cosine_precision@

In [151]:
def cosine_similarit(vec1, vec2):
    """Calculate the cosine similarity between two vectors."""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)


# model = AutoModel.from_pretrained("classla/bcms-bertic")
# # Combine the model and pooling into a SentenceTransformer
# word_embedding_model = models.Transformer(model_name_or_path="classla/bcms-bertic")
# pooling_model = models.Pooling(word_embedding_dimension=model.config.hidden_size, pooling_mode_mean_tokens=True)
# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
# # model = SentenceTransformer("classla/bcms-bertic", device="cpu")

# tokens1 = model[0].tokenize("")
# res1 = model[0].forward(tokens1)["token_embeddings"][0][1].detach().numpy()

# tokens2 = model[0].tokenize("covek jase konja")
# res2 = model[0].forward(tokens2)["token_embeddings"][0][1].detach().numpy()
# res2
# cosine_similarity(res1, res2)
# cosine_similarity(np.array(res1).reshape(1, -1), np.array(res2).reshape(1, -1))

In [132]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained model tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained model
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()  # Set model to evaluation mode

# Tokenize input text
text = "This is an example sentence for BERT embeddings."
inputs = tokenizer(text, return_tensors='pt')

# Get embeddings
with torch.no_grad():
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state  # Embeddings

# The last_hidden_states tensor contains the embeddings for each token
print(last_hidden_states)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

tensor([[[-0.3137, -0.3394, -0.3190,  ..., -0.6073,  0.0885,  0.8370],
         [-0.7485, -0.5191, -0.5937,  ..., -0.3184,  0.7032,  0.2263],
         [-0.4606, -0.7230,  0.0426,  ..., -0.0821,  0.3424,  0.7808],
         ...,
         [-0.3740, -0.2744, -0.3737,  ..., -0.3573, -1.0819,  0.4773],
         [ 0.6473,  0.1670, -0.6649,  ...,  0.3043, -0.6731, -0.3639],
         [ 0.5446,  0.1987, -0.4165,  ...,  0.5330, -1.0449, -0.2541]]])


In [ ]:
def get_ebm(text):
    inputs = tokenizer(text, return_tensors='pt')

    # Get embeddings
    with torch.no_grad():
        outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state  # Embeddings

    # The last_hidden_states tensor contains the embeddings for each token
    return last_hidden_states

In [1]:
from sentence_transformers import SentenceTransformer

# Load pre-trained sentence transformer model
model = SentenceTransformer('jerteh/Jerteh-81')

# Encode a list of sentences
sentences = ["This is an example sentence for BERT embeddings."]
sentence_embeddings = model.encode(sentences)

# print(sentence_embeddings)

/home/selena/sele/SRBedding/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
No sentence-transformers model found with name jerteh/Jerteh-81. Creating a new one with mean pooling.
Some weights of RobertaModel were not initialized from the model checkpoint at jerteh/Jerteh-81 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/selena/sele/SRBedding/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` 

In [2]:
sentence1 = "this is a great laptop"
sentence2 = "yesterday I saw Marrie at the park."

encoded_input1 = model.encode(sentence1)
encoded_input2 = model.encode(sentence2)
print(len(encoded_input1))


# Reshape the embeddings to 2D arrays
encoded_input1 = np.array(encoded_input1).reshape(1, -1)
encoded_input2 = np.array(encoded_input2).reshape(1, -1)

# Calculate cosine similarity
similarity = cosine_similarity(encoded_input1, encoded_input2)[0][0]

print(f"Cosine Similarity: {similarity:.4f}")


768


NameError: name 'np' is not defined

In [177]:
sentence1 = "Glavni grad Srbije"
sentence2 = "Macka je siva"

encoded_input1 = model.encode(sentence1)
encoded_input2 = model.encode(sentence2)

# Reshape the embeddings to 2D arrays
encoded_input1 = np.array(encoded_input1).reshape(1, -1)
encoded_input2 = np.array(encoded_input2).reshape(1, -1)

# Calculate cosine similarity
similarity = cosine_similarity(encoded_input1, encoded_input2)[0][0]

print(f"Cosine Similarity: {similarity:.4f}")


Cosine Similarity: 0.5224


In [196]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from torch import LongTensor, no_grad
from scipy import spatial
tokenizer = AutoTokenizer.from_pretrained('jerteh/jerteh-81')
model = AutoModelForMaskedLM.from_pretrained('jerteh/jerteh-81', output_hidden_states=True)
x = " pas"
y = " mačka"
z = " jazavicar"
tensor_x = LongTensor(tokenizer.encode(x, add_special_tokens=False)).unsqueeze(0)
tensor_y = LongTensor(tokenizer.encode(y, add_special_tokens=False)).unsqueeze(0)
tensor_z = LongTensor(tokenizer.encode(z, add_special_tokens=False)).unsqueeze(0)
model.eval()
with no_grad():
    vektor_x = model(input_ids=tensor_x).hidden_states[-1].squeeze()
    vektor_y = model(input_ids=tensor_y).hidden_states[-1].squeeze()
    vektor_z = model(input_ids=tensor_z).hidden_states[-1].squeeze()
    print(spatial.distance.cosine(vektor_x, vektor_y))
    print(spatial.distance.cosine(vektor_x, vektor_z))

0.09954953


ValueError: Input vector should be 1-D.

In [197]:
x = " pas je velik"
y = " mačka je velika"
z = " svemir je velik"
tensor_x = LongTensor(tokenizer.encode(x, add_special_tokens=False)).unsqueeze(0)
tensor_y = LongTensor(tokenizer.encode(y, add_special_tokens=False)).unsqueeze(0)
tensor_z = LongTensor(tokenizer.encode(z, add_special_tokens=False)).unsqueeze(0)
model.eval()
with no_grad():
    vektor_x = model(input_ids=tensor_x).hidden_states[-1].squeeze()
    vektor_y = model(input_ids=tensor_y).hidden_states[-1].squeeze()
    vektor_z = model(input_ids=tensor_z).hidden_states[-1].squeeze()
    print(spatial.distance.cosine(vektor_x, vektor_y))
    print(spatial.distance.cosine(vektor_x, vektor_z))

ValueError: Input vector should be 1-D.

In [191]:
x = " pas"
y = " mačka"
z = " svemir"
tensor_x = LongTensor(tokenizer.encode(x, add_special_tokens=False)).unsqueeze(0)
tensor_y = LongTensor(tokenizer.encode(y, add_special_tokens=False)).unsqueeze(0)
tensor_z = LongTensor(tokenizer.encode(z, add_special_tokens=False)).unsqueeze(0)
model.eval()
with no_grad():
    vektor_x = model(input_ids=tensor_x).hidden_states[-1].squeeze()
    vektor_y = model(input_ids=tensor_y).hidden_states[-1].squeeze()
    vektor_z = model(input_ids=tensor_z).hidden_states[-1].squeeze()
    print(spatial.distance.cosine(vektor_x, vektor_y))
    print(spatial.distance.cosine(vektor_x, vektor_z))

0.09954953
0.21845555


In [200]:
import torch
from torch import no_grad, LongTensor
from transformers import BertTokenizer, BertModel
from scipy.spatial import distance

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

# Define sentences
x = "pas se igra sa teniskom lopticom"
y = "macka sedi."
# z = "The universe is vast and mysterious."
z = "pas se igra sa teniskom lopticom."


# Tokenize and encode sentences
tensor_x = LongTensor(tokenizer.encode(x, add_special_tokens=True)).unsqueeze(0)
tensor_y = LongTensor(tokenizer.encode(y, add_special_tokens=True)).unsqueeze(0)
tensor_z = LongTensor(tokenizer.encode(z, add_special_tokens=True)).unsqueeze(0)

# Generate embeddings
with no_grad():
    vektor_x = model(input_ids=tensor_x).last_hidden_state.mean(dim=1).squeeze()
    vektor_y = model(input_ids=tensor_y).last_hidden_state.mean(dim=1).squeeze()
    vektor_z = model(input_ids=tensor_z).last_hidden_state.mean(dim=1).squeeze()

# Calculate cosine similarities
cos_sim_xy = distance.cosine(vektor_x.numpy(), vektor_y.numpy())
cos_sim_xz = distance.cosine(vektor_x.numpy(), vektor_z.numpy())

print(f"Cosine similarity between x and y: {1 - cos_sim_xy}")
print(f"Cosine similarity between x and z: {1 - cos_sim_xz}")


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Cosine similarity between x and y: 0.6164170503616333
Cosine similarity between x and z: 0.9530267715454102


In [1]:
name = "marco_processed.parquet".split("_")[0]
name

'marco'